In [1]:
import sys
print(sys.executable)

C:\Users\oswal\OneDrive\Documentos\GitHub\Neurociencias-2025-2\dl_env\Scripts\python.exe


In [2]:
import sys
print(sys.version)

3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]


In [3]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [4]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [6]:
#Crear un entorno virtual para nuestro work space
# cd es change directori, y se pega la dirección a la que se quiere cambiar y se pone sin comillas


In [8]:
path = r"C:\Users\oswal\OneDrive\Documentos\GitHub\Neurociencias-2025-2\S03_datasets\iris\iris.csv"

df = pd.read_csv(path)
df.head(3)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [9]:
X_, y = df.iloc[:, :-1], df.iloc[:, -1].astype('category').cat.codes

# Normalizando datos
X = StandardScaler().fit_transform(X_)

In [10]:
y_cat = to_categorical(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size = 0.2, random_state = 0)

In [12]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

In [13]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 10, activation = 'relu', input_shape=(4,)))

# Adding the output layer
classifier.add(Dense(units = 3, activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test))

Epoch 1/150
4/4 [==============================] - 3s 188ms/step - loss: 1.2633 - accuracy: 0.0417 - val_loss: 1.2591 - val_accuracy: 0.0667
Epoch 2/150
4/4 [==============================] - 0s 22ms/step - loss: 1.2465 - accuracy: 0.0417 - val_loss: 1.2429 - val_accuracy: 0.0667
Epoch 3/150
4/4 [==============================] - 0s 17ms/step - loss: 1.2306 - accuracy: 0.0417 - val_loss: 1.2273 - val_accuracy: 0.0667
Epoch 4/150
4/4 [==============================] - 0s 33ms/step - loss: 1.2147 - accuracy: 0.0583 - val_loss: 1.2128 - val_accuracy: 0.1333
Epoch 5/150
4/4 [==============================] - 0s 34ms/step - loss: 1.1992 - accuracy: 0.1083 - val_loss: 1.1982 - val_accuracy: 0.1333
Epoch 6/150
4/4 [==============================] - 0s 19ms/step - loss: 1.1845 - accuracy: 0.1500 - val_loss: 1.1840 - val_accuracy: 0.2000
Epoch 7/150
4/4 [==============================] - 0s 22ms/step - loss: 1.1701 - accuracy: 0.1917 - val_loss: 1.1698 - val_accuracy: 0.2333
Epoch 8/150
4/4 [==

In [14]:
# 5. Evaluar
y_pred = classifier.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\n🔎 Reporte de clasificación:")
print(classification_report(y_true, y_pred_classes, target_names=df.Species.unique()))

1/1 [==============================] - 0s 207ms/step

🔎 Reporte de clasificación:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        11
  versicolor       1.00      0.69      0.82        13
   virginica       0.60      1.00      0.75         6

    accuracy                           0.87        30
   macro avg       0.87      0.90      0.86        30
weighted avg       0.92      0.87      0.87        30



In [15]:
# Evaluating the ANN
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

In [16]:
def build_classifier():
    # Initialising the ANN
    classifier = Sequential()

    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units = 10, activation = 'relu', input_shape=(4,)))

    # Adding the output layer
    classifier.add(Dense(units = 3, activation = 'softmax'))

    # Compiling the ANN
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    return classifier

In [17]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

mean = accuracies.mean()
variance = accuracies.std()

In [18]:
mean, variance

(0.8916666666666666, 0.07499999999999998)

In [25]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
# Tuning the ANN
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV


In [20]:
def build_classifier(optimizer='adam'):
    classifier = Sequential()
    classifier.add(Dense(units = 10, activation = 'relu', input_shape=(4,)))
    classifier.add(Dense(units = 3, activation = 'softmax'))
    classifier.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return classifier

In [21]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

mean = accuracies.mean()
variance = accuracies.std()
mean, variance

(0.9166666666666666, 0.07453559924999298)

In [23]:
classifier = KerasClassifier(model = build_classifier, verbose=0)
parameters = {'batch_size': [10],
              'epochs': [100],
              'optimizer': ['adam']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

grid_search.fit(X = X_train, y = y_train)


GridSearchCV(cv=10,
             estimator=KerasClassifier(model=<function build_classifier at 0x000001F2798CB9A0>, verbose=0),
             param_grid={'batch_size': [10], 'epochs': [100],
                         'optimizer': ['adam']},
             scoring='accuracy')

In [28]:
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'batch_size': 10, 'epochs': 100, 'optimizer': 'adam'}
Best score: 0.9083333333333332
